<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
تشخیص جنسیت
</font>
</h1>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    امروزه شبکه‌های اجتماعی کاربردهای گسترده‌ای دارند. اولین استفاده آن، تفریح و اوقات فراغت است. اما از دید دیگر، می‌توان از شبکه‌های اجتماعی برای پیدا کردن الگوهای رفتاری استفاده کرد. به عنوان مثال با تحلیل نظرات کاربران شبکه‌های اجتماعی می‌توانیم ضعف‌های کسب و کار خود را پیدا کنیم.
    <br>
    جنسیت یکی از پارامترهای تاثیرگذار در رفتار کاربران است. در مواجه با یک موضوع، بانوان عمدتا یک‌طور واکنش نشان خواهند داد و آقایان نیز طور دیگری!
    <br>
    حال در این تمرین قصد داریم با اطلاعاتی که دیتاک از کاربران توییتر و اینستاگرم در اختیار ما قرار داده است، جنسیت آن‌ها را پیش‌بینی کنیم.
</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ابتدا کتابخانه‌های مورد نیازتان را وارد کنید.
</font>
</p>

In [1]:
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from hazm import *

import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


import scipy.sparse as ss

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, f1_score


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
مجموعه داده آموزش ۸۰۰۰ سطر و ۱۰ ستون دارد. در جدول زیر، اطلاعات بیشتری در مورد داده نوشته شده است.     
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|gender|جنسیت (ستون هدف)|
|age|بازه سنی کاربر|
|fullname|نامی که در پروفایل شبکه اجتماعی نوشته شده است |
|username|نام کاربری|
|biography|بیوگرافی شبکه اجتماعی کاربر|
|follower_count|تعداد دنبال‌کننده‌های کاربر|
|following_count|تعداد افرادی که کاربر را دنبال می‌کنند|
|is_business|کسب‌وکاری بودن حساب کاربری|
|is_verified|تایید شده بودن حساب کاربری|
|is_private|خصوصی بودن حساب کاربری|
    
</font>
</div>
</center>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ستون <code>age</code> متغیر پیوسته نیست؛ بلکه دسته‌های سنی را نشان می‌دهد. در جدول زیر نحوه نگاشت سن را مشاهده می‌کنیم.
</font>
</p>


<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|سن واقعی کاربران|عدد نگاشت شده|
|:------:|:---:|
|1|کمتر از ۱۸ سال|
|2|از ۱۹ تا ۲۹ سال|
|3|از ۳۰ تا ۴۰ سال|
|4|بالاتر از ۴۰ سال|
    
</font>
</div>
</center>


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در ابتدا نیاز است فایل‌های مجموعه‌داده را بخوانید. نمونه‌های آموزشی در فایل <code>train_data.csv</code> و نمونه‌های آزمون که باید دسته‌ی آن‌ها را پیش‌بینی کنید در فایل <code>test_data.csv</code> ذخیره شده‌اند. اگر لازم دانستید می‌توانید به دلخواه خود بخشی از دادگان آموزشی را به عنوان دادگان اعتبارسنجی نیز جدا کنید.
</font>
</p>

In [2]:
train = pd.read_csv('../data/train_data.csv')
test = pd.read_csv('../data/test_data.csv')
train

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
0,man,2,Farshid,mr_gh_farshid,دردا ک در این بادیه بسیار دویدیم...\nGlory man...,1604.0,1407.0,0.0,0.0,0.0
1,woman,2,zahr@72,zahra.roozbahani72,"خواهی که زکوچ در امان برگردی\nباید که به جان ,...",67.0,501.0,0.0,0.0,0.0
2,woman,2,ms farahnaz♥,___lady.farahnazi.__,"Having you, is all I wish for \nداشتنت، تمامِ...",0.0,0.0,0.0,0.0,0.0
3,woman,1,Lena.mommy farzan,mommy.lena3361,دردونه من لنا کوچولو,0.0,0.0,0.0,0.0,0.0
4,woman,2,Narsis Asadollahi,_l.aurora.l_,I am an animation student\n🎧🎼🎨⚓️🤍 \n@general.m...,200.0,328.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7995,woman,2,Ŋεgɨŋ,negiiin_bahrmandi,﷽\nAllah IS Enough FoR Me?\n♥️?,0.0,0.0,0.0,0.0,0.0
7996,man,3,h🗯abdi🗯offìcial,h.abdi.official,حقوقی,0.0,0.0,1.0,0.0,0.0
7997,woman,2,⚜رویا احمدی⚜,roya.ahmadi.k,مهندس صنایع👩‍🔧🏭 Industrial engineer\nمعمار👩‍💻👷...,1260.0,1167.0,0.0,0.0,0.0
7998,man,3,لرستان &خرم اباد,erfanpouersif,khoramabad,0.0,0.0,0.0,0.0,0.0


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی که در فصل‌های گذشته آموختید، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.
</font>
</p>

In [3]:
train.describe(include='all')

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
count,8000,8000.000000,8000,8000,8000,8000.000000,8000.00000,7997.000000,8000.000000,8000.000000
unique,2,NaN,7171,8000,7930,NaN,NaN,NaN,NaN,NaN
top,man,NaN,maryam,mr_gh_farshid,فقط خدا,NaN,NaN,NaN,NaN,NaN
freq,4000,NaN,28,1,5,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.659875,NaN,NaN,NaN,2745.200250,712.20650,0.103289,0.000125,0.000125
std,NaN,0.724228,NaN,NaN,NaN,20156.322638,1465.37601,0.304355,0.011180,0.011180
min,NaN,1.000000,NaN,NaN,NaN,0.000000,0.00000,0.000000,0.000000,0.000000
25%,NaN,2.000000,NaN,NaN,NaN,0.000000,0.00000,0.000000,0.000000,0.000000
50%,NaN,3.000000,NaN,NaN,NaN,124.000000,133.00000,0.000000,0.000000,0.000000
75%,NaN,3.000000,NaN,NaN,NaN,917.250000,681.00000,0.000000,0.000000,0.000000


In [4]:
# do some preprocessing :)
train.nunique()

gender                2
age                   4
fullname           7171
username           8000
biography          7930
follower_count     2477
following_count    2055
is_business           2
is_verified           2
is_private            2
dtype: int64

In [5]:
train[train['is_verified'] == 1].describe(include='all')

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
count,1,1.0,1,1,1,1.0,1.0,1.0,1.0,1.0
unique,1,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN
top,woman,NaN,Nooshafarin نوش آفرين,nooshafarin.a,Nooshafarin official page\nصفحه رسمی نوش افرین,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,4.0,NaN,NaN,NaN,296244.0,118.0,1.0,1.0,0.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,4.0,NaN,NaN,NaN,296244.0,118.0,1.0,1.0,0.0
25%,NaN,4.0,NaN,NaN,NaN,296244.0,118.0,1.0,1.0,0.0
50%,NaN,4.0,NaN,NaN,NaN,296244.0,118.0,1.0,1.0,0.0
75%,NaN,4.0,NaN,NaN,NaN,296244.0,118.0,1.0,1.0,0.0


In [6]:
train[train['is_business'] == 1].describe(include='all')

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
count,826,826.000000,826,826,826,826.000000,826.000000,826.0,826.000000,826.0
unique,2,NaN,817,826,826,NaN,NaN,NaN,NaN,NaN
top,man,NaN,alireza,officemehran,وی در خانواده مذهبی اعتقاد خود را از دست داد😀😀,NaN,NaN,NaN,NaN,NaN
freq,524,NaN,3,1,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.742131,NaN,NaN,NaN,11711.389831,1532.726392,1.0,0.001211,0.0
std,NaN,0.725362,NaN,NaN,NaN,46122.635507,2156.302881,0.0,0.034794,0.0
min,NaN,1.000000,NaN,NaN,NaN,0.000000,0.000000,1.0,0.000000,0.0
25%,NaN,2.000000,NaN,NaN,NaN,800.750000,129.000000,1.0,0.000000,0.0
50%,NaN,3.000000,NaN,NaN,NaN,2583.500000,614.000000,1.0,0.000000,0.0
75%,NaN,3.000000,NaN,NaN,NaN,8935.500000,1712.250000,1.0,0.000000,0.0


In [7]:
train.drop(columns=['follower_count','following_count', 'is_verified','is_private','username', 'age','biography'],inplace=True)
test.drop(columns=['follower_count','following_count', 'is_verified','is_private','username','age', 'biography'],inplace=True)

In [8]:
train

,gender,fullname,is_business
0,man,Farshid,0.0
1,woman,zahr@72,0.0
2,woman,ms farahnaz♥,0.0
3,woman,Lena.mommy farzan,0.0
4,woman,Narsis Asadollahi,0.0
...,...,...,...
7995,woman,Ŋεgɨŋ,0.0
7996,man,h🗯abdi🗯offìcial,1.0
7997,woman,⚜رویا احمدی⚜,0.0
7998,man,لرستان &خرم اباد,0.0


In [9]:
train.isna().sum()

gender         0
fullname       0
is_business    3
dtype: int64

In [10]:
train.is_business.fillna(0.0, inplace=True)

In [11]:
import string
from hazm import stopwords_list
from nltk.tokenize.casual import TweetTokenizer

normalizer = Normalizer(persian_numbers =False,persian_style=False)
tokenizer = WordTokenizer(separate_emoji=True , replace_links=True ,replace_ids=True ,replace_emails=False ,replace_numbers=True ,replace_hashtags=True)
stop_words_EN = set(stopwords.words('english'))
stop_words_FA = set(stopwords_list())
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
t = TweetTokenizer()

# 
# A = -1
def pre_fast(text):
	

	
	#lowercase_text(text)
	text = text.lower()

	text = re.sub(r'\d+', '', text)

	#remove_punctuation(text):
	text = text.translate(str.maketrans('', '', string.punctuation))

	punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~؟!.،,?'''

	# Removing punctuations in string
	# Using loop + punctuation string
	for ele in text:
		if ele in punc:
			text = text.replace(ele, "")

	punc = '''\u200d♂️٪'🏻','''
	# Removing punctuations in string
	# Using loop + punctuation string
	for ele in text:
		if ele in punc:
			text = text.replace(ele, "")

	text = normalizer.normalize(text)


	# Remove_stop_words_English and Remove_stop_words_Farsi and Lemmatizer_Farsi and Stemmer_English
	#romove number


	text = re.sub(r'\d+', '', text)
	words = tokenizer.tokenize(text)
	text = ' '.join(words)
	words = t.tokenize(text)


	
	# words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in (stop_words_EN | stop_words_FA)]

	words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words if word not in (stop_words_EN | stop_words_FA)]

	text = ' '.join(words)




	punc = '''\u200c'''

	# Removing punctuations in string
	# Using loop + punctuation string
	for ele in text:
		if ele in punc:
			text = text.replace(ele, " ")

	
	text = re.sub(r'\d+', '', text)
	words = tokenizer.tokenize(text)
	text = ' '.join(words)
	words = t.tokenize(text)
	words = t.tokenize(text)
	# words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in (stop_words_EN | stop_words_FA)]

	words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words if word not in (stop_words_EN | stop_words_FA)]

	text = ' '.join(words)

	text = normalizer.normalize(text)
	#remove_punctuation(text):
	text = text.translate(str.maketrans('', '', string.punctuation))

	text = text.replace('''\u200c''', " ")
	text = text.replace('\u200c', " ")
	
	global A
	# print(A)
	# A +=1

	return	text



In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer


corpus =  train['fullname']
tfidf_vect_fullname = TfidfVectorizer(preprocessor =pre_fast,min_df = 2)
final_counts_fullname = tfidf_vect_fullname.fit_transform(corpus)

print(tfidf_vect_fullname.get_feature_names_out())
print(final_counts_fullname.toarray()[0])
print(final_counts_fullname.toarray()[1])

['aba' 'abadi' 'abba' ... '乙ﾑ' '𝓐𝓶𝓲𝓷' '𝓢𝓪𝓶𝓲𝓻𝓪']
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


In [13]:
tfidf_vect_fullname.get_feature_names_out()

array(['aba', 'abadi', 'abba', ..., '乙ﾑ', '𝓐𝓶𝓲𝓷', '𝓢𝓪𝓶𝓲𝓻𝓪'], dtype=object)

In [14]:

test_biography = test['fullname']
test_counts_fullname = tfidf_vect_fullname.transform(test_biography)


In [15]:
final_counts_fullname.toarray().shape

(8000, 1203)

In [16]:
test_counts_fullname.toarray().shape

(2000, 1203)

In [17]:
train_data = pd.DataFrame(final_counts_fullname.toarray())
test_data = pd.DataFrame(test_counts_fullname.toarray())
train_data

,0,1,2,3,4,5,6,7,8,9,...,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7996,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
train.drop(columns= ['fullname'],inplace=True)

train['gender'].replace({'woman':0, 'man':1},inplace=True)
train

,gender,is_business
0,1,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
7995,0,0.0
7996,1,1.0
7997,0,0.0
7998,1,0.0


In [19]:
test.drop(columns= ['fullname'],inplace=True)


In [20]:
concatenated_test_data= pd.concat([test, test_data],axis=1)
concatenated_test_data.columns = concatenated_test_data.columns.astype(str)
concatenated_test_data

,is_business,0,1,2,3,4,5,6,7,8,...,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
concatenated_train_data = pd.concat([train, train_data],axis=1)
concatenated_train_data.columns = concatenated_train_data.columns.astype(str)
concatenated_train_data

,gender,is_business,0,1,2,3,4,5,6,7,...,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7996,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7997,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7998,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
concatenated_train_data['is_business'] = concatenated_train_data['is_business']/4
concatenated_test_data['is_business'] = concatenated_test_data['is_business']/4
concatenated_test_data

,is_business,0,1,2,3,4,5,6,7,8,...,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202
0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    حال که داده را پاکسازی کرده و احتمالا ویژگی‌هایی را به آن افزوده یا از آن حذف کرده‌اید، وقت آن است که مدلی آموزش دهید که بتواند متغیر هدف این مسئله را پیش‌بینی کند.
</font>
</p>

In [23]:
from sklearn.model_selection import train_test_split
X = concatenated_train_data.drop(columns=['gender'])
y = concatenated_train_data['gender']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [24]:
# modeling :)
from sklearn.neural_network import MLPClassifier



model = MLPClassifier()

model.fit(X_train, y_train)


MLPClassifier()

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>f1_score</code> نام دارد.
    <br>
    این معیار، سنجه ارزیابی کیفیت مدل شماست. به عبارت بهتر در سامانه داوری هم از همین معیار برای نمره‌دهی استفاده شده است.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی دادگان آموزش یا اعتبارسنجی ارزیابی کنید.
</font>
</p>

In [25]:
y_pred = model.predict(X_test)

In [26]:
# evaluate model
from sklearn.metrics import classification_report, f1_score

f1 = f1_score(y_test, y_pred, average='weighted') 

print("F1 Score:", f1)

class_report = classification_report(y_test, y_pred)

print("Classification Report:")
print(class_report)

F1 Score: 0.8051693121693122
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.94      0.83       597
           1       0.92      0.68      0.78       603

    accuracy                           0.81      1200
   macro avg       0.83      0.81      0.81      1200
weighted avg       0.83      0.81      0.81      1200



<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
 جهت کسب امتیاز کامل نیاز است تا پاسخ شما حداقل امتیاز <code>75%</code> را با توجه به معیار معرفی‌شده کسب نماید.
</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    پس از مهندسی ویژگی و مدلسازی، الگوریتمی دارید که می‌تواند شما را از متغیرهای مستقل به متغیر هدف برساند.
    <br>
    از این مدل برای پیش‌بینی نمونه‌های موجود در داده تست استفاده کنید و نتایج را در قالب جدول (<code>dataframe</code>) زیر آماده کنید.
</font>
</p>

<div dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|gender|جنسیت کاربر|
    
</font>
</div>



<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    اسم دیتافریم باید <i>submission</i> باشد؛ در غیر این صورت، سامانه داوری نمی‌تواند تلاش‌ شما را ارزیابی کند.
    <br>
    این دیتافریم تنها شامل ۱ ستون با اسم <i>gender</i> است و ۲۰۰۰ سطر دارد.
    <br>
    به ازای هر سطر موجود در دیتافریم <i>test_data</i> شما باید یک مقدار پیشبینی شده داشته باشید.
    <br>
    جدول زیر، ۵ سطر ابتدایی دیتافریم <code>submission</code> را نشان می‌دهد. البته در جواب شما، مقادیر ستون <i>gender</i> ممکن است متفاوت باشد.
</font>
</p>

<div style="text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
||gender|
|----|-----|
|0|woman|
|1|woman|
|2|man|
|3|woman|
|4|man|

</font>
</div>



In [27]:
pred = model.predict(concatenated_test_data)


In [28]:
# predict test samples
submission = pd.DataFrame(pred,columns=['gender'])
submission['gender'].replace({0:'woman', 1:'man'},inplace=True)

submission

,gender
0,woman
1,man
2,man
3,woman
4,woman
...,...
1995,woman
1996,woman
1997,woman
1998,woman


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>

In [29]:
import zipfile
import joblib

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['gender_detection.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['gender_detection.ipynb', 'submission.csv']
